## Machine Learning : Optimisation de Code et Clustering Avancé

Ce cours de 3 heures explore les aspects avancés de l'optimisation de code Python et l'algorithme de clustering DBSCAN, offrant une approche approfondie des techniques d'amélioration des performances et des méthodes de clustering basées sur la densité.

Lien vers l'audio : https://audio-records-dsfs.s3.eu-west-3.amazonaws.com/UnSupervisedML/M07D02_DataScience.m4a

## **Introduction et Contexte Pédagogique**

L'approche du cours suit une philosophie claire : **"S'il y a une boucle forte, on essaie de l'enlever"**. Cette maxime reflète l'importance de l'optimisation en machine learning, particulièrement cruciale lorsque l'on travaille avec de grandes quantités de données.[1]

Le cours combine théorie et pratique en se basant sur des **exemples concrets d'optimisation** observés en temps réel, démontrant comment transformer du code fonctionnel mais lent en solutions performantes.

## **Optimisation Fondamentale du Code Python**

### **Problématiques des Boucles Traditionnelles**

Les boucles Python présentent des limitations intrinsèques :[2][3]

**Surcoût d'interprétation** : Python interprète chaque itération individuellement, créant une overhead computationnelle significative. Pour 1000 points et 3 centroïdes, cela représente 3000 calculs de distance par itération.[1]

**Masquage répétitif** : L'application de filtres (masques) de manière répétée sur les mêmes données devient particulièrement coûteuse. Chaque `mask = (cluster_labels == label)` doit parcourir l'intégralité du dataset.[1]

### **Solution : Module Collections**

Le module `collections.Counter` révolutionne le comptage d'occurrences. Contrairement aux dictionnaires traditionnels, Counter offre des méthodes spécialisées :[4][5]

**most_common(n)** : Extrait les n éléments les plus fréquents en temps linéaire. Cette méthode évite le tri complet et l'indexation manuelle.[6]

**Opérations arithmétiques** : Permet l'addition et la soustraction directe entre Counter, facilitant les analyses comparatives.[4]

### **Transformation Pratique d'Optimisation**

**Version initiale (lente)** :
```python
# Approche avec boucles explicites - O(k*n)
for cluster_id in unique_clusters:
    mask = (cluster_labels == cluster_id)  # Coûteux
    target_counts = data[mask]['target'].value_counts()
    most_frequent_target = target_counts.index[0]
    cluster_mapping[cluster_id] = most_frequent_target
```

**Version optimisée** :
```python
# Groupby + Counter - O(n)
grouped_data = df.groupby('cluster_label')['target'].apply(Counter)
optimized_mapping = {
    cluster: counter.most_common(1)[0][0] 
    for cluster, counter in grouped_data.items()
}
```

Les **tests de performance** réalisés en cours montrent une **amélioration de 5x** sur des datasets volumineux, avec une complexité réduite de O(k*n) à O(n).[1]

## **Révision Approfondie de K-means**

### **Mécanisme Algorithmique Détaillé**

L'algorithme K-means suit un processus itératif rigoureux :[1]

1. **Initialisation aléatoire** des K centroïdes dans l'espace des features
2. **Attribution** : Calcul de 3000 distances pour 1000 points et 3 centroïdes
3. **Recalcul** : Centroïdes repositionnés à la moyenne de leurs clusters
4. **Convergence** : Répétition jusqu'à stabilisation ou limite d'itérations

### **WCSS - Compréhension Mathématique**

La **Within Cluster Sum of Squares** suit une propriété mathématique fondamentale :[7][8]

$$ \text{WCSS} = \sum_{i=1}^{k} \sum_{j \in C_i} ||x_j - \mu_i||^2 $$

**Propriété critique** : WCSS décroît mécaniquement avec k. Avec k=nombre de points, WCSS=0 car chaque point devient son propre centroïde. Cette propriété explique pourquoi la méthode du coude recherche un **changement de pente significatif** plutôt qu'une valeur minimale absolue.[1]

### **Silhouette Score - Interprétation Avancée**

Le calcul du Silhouette Score pour chaque point i :[9][10]

$$ s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))} $$

**Points en bordure** : Lorsqu'un point se situe à la frontière entre clusters, son score peut devenir négatif. Cela ne signifie pas nécessairement une mauvaise assignation, mais plutôt une **proximité inter-clusters**.[1]

**Silhouette du modèle** : La moyenne arithmétique des scores individuels, pénalisée par les points en bordure ayant des valeurs négatives.

## **DBSCAN - Clustering par Densité**

### **Limitations Structurelles de K-means**

K-means crée invariablement des **partitions sphériques** autour des centroïdes. Cette limitation devient évidente avec des données en forme de :[11]
- **Croissants de lune** (make_moons dataset)[12][13]
- **Clusters allongés** ou de formes arbitraires
- **Densités variables** dans l'espace des features

### **Paradigme de Densité**

DBSCAN révolutionne l'approche en se basant sur la **densité locale** plutôt que sur la proximité à un centre. Cette approche permet de :[14][11]
- Détecter des clusters de **formes arbitraires**
- Identifier automatiquement les **outliers**
- Adapter le clustering à la **structure intrinsèque** des données

### **Taxonomie des Points**

**Core Points** : Points possédant ≥ min_samples voisins dans un rayon ε. Ces points forment la **colonne vertébrale** des clusters.[15][11]

**Border Points** : Points avec < min_samples voisins mais situés dans le rayon ε d'un core point. Ils **appartiennent au cluster** mais ne peuvent pas l'étendre.[11]

**Noise Points** : Points isolés n'appartenant à aucun cluster, étiquetés -1. Représentent les **outliers naturels** du dataset.[11]

### **Processus Algorithmique**

L'algorithme suit une **expansion de proche en proche** :[1]

1. **Sélection** d'un point non-visité
2. **Comptage** des voisins dans le rayon ε
3. **Classification** : Core, Border, ou potentiel Noise
4. **Expansion** : Propagation du cluster vers tous les points density-reachable
5. **Itération** jusqu'à épuisement des points

**Exemple concret** avec ε=0.2, min_samples=4 :[1]
- Point A : 5 voisins → Core point → Initie Cluster 1
- Point B : 3 voisins mais dans rayon de A → Border point → Rejoint Cluster 1
- Point C : 2 voisins, isolé → Noise point → Étiquette -1

### **Optimisation des Paramètres**

#### **Méthode k-distance**

La détermination optimale d'epsilon utilise l'analyse des **k-plus-proches-voisins** :[16][17]

```python
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors.fit(X_scaled)
distances, indices = neighbors.kneighbors(X_scaled)

# k-distances triées
k_distances = np.sort(distances[:, min_samples-1])
```

**Interprétation graphique** :[17]
- **Plateau initial** : Points dans des clusters denses
- **Coude prononcé** : Transition vers les outliers  
- **Montée abrupte** : Points complètement isolés

L'epsilon optimal se situe généralement **au niveau du coude**, représentant la distance caractéristique intra-cluster.

#### **Impact Paramétrique**

**ε trop petit** :[18][1]
- Fragmentation excessive des clusters
- Classification de nombreux points comme noise
- Perte d'informations structurelles

**ε trop grand** :[1]
- Fusion de clusters distincts
- Réduction artificielle du nombre de groupes
- Possible création d'un unique mega-cluster

**min_samples critique** :[1]
- **Trop petit** (1-2) : Sensibilité excessive au bruit
- **Trop grand** (>10% du dataset) : Seules les zones très denses forment des clusters

## **Exemples Pratiques et Implémentation**

### **Dataset make_moons**

Le dataset make_moons génère **deux croissants entrelacés**, parfait pour illustrer les limitations de K-means :[13][12]

```python
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=300, noise=0.1, random_state=42)
```

**Paramètres cruciaux** :
- **noise** : Contrôle la dispersion gaussienne (0.05 = peu de bruit, 0.5 = très dispersé)
- **n_samples** : Nombre total de points (distribution équitable sur les deux croissants)

### **Comparaison Algorithmique**

**K-means sur make_moons** : Crée invariablement une séparation linéaire inadéquate, ignorant la courbure naturelle des croissants.[1]

**DBSCAN sur make_moons** : S'adapte parfaitement à la forme courbe, respectant la structure géométrique intrinsèque des données.

### **Optimisation Avancée avec NearestNeighbors**

```python
def analyze_epsilon_range(X, min_samples_target):
    """
    Analyse les distances pour suggérer une plage d'epsilon pertinente
    """
    neighbors = NearestNeighbors(n_neighbors=min_samples_target)
    neighbors.fit(X)
    distances, _ = neighbors.kneighbors(X)
    
    k_distances = distances[:, min_samples_target-1]
    
    # Statistiques descriptives pour guider le choix
    percentiles = np.percentile(k_distances, [25, 50, 75, 90])
    
    return {
        'min_distance': k_distances.min(),
        'median_distance': percentiles[1], 
        'suggested_range': (percentiles[0], percentiles[2]),
        'noise_threshold': percentiles[3]
    }
```

Cette analyse **évite les tests aléatoires** d'epsilon en fournissant une plage statistiquement fondée.[1]

### **Évaluation Comparative**

```python
def comprehensive_clustering_comparison(X, true_labels=None):
    """
    Comparaison exhaustive K-means vs DBSCAN
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # K-means configuration
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    kmeans_labels = kmeans.fit_predict(X_scaled)
    
    # DBSCAN avec paramètres optimisés
    epsilon_analysis = analyze_epsilon_range(X_scaled, 5)
    optimal_eps = epsilon_analysis['median_distance']
    
    dbscan = DBSCAN(eps=optimal_eps, min_samples=5)
    dbscan_labels = dbscan.fit_predict(X_scaled)
    
    # Métriques comparatives
    metrics = {}
    
    if true_labels is not None:
        from sklearn.metrics import adjusted_rand_score, homogeneity_score
        metrics['K-means_ARI'] = adjusted_rand_score(true_labels, kmeans_labels)
        metrics['DBSCAN_ARI'] = adjusted_rand_score(true_labels, dbscan_labels)
    
    # Silhouette scores
    metrics['K-means_Silhouette'] = silhouette_score(X_scaled, kmeans_labels)
    
    # DBSCAN silhouette uniquement si clusters valides
    if len(set(dbscan_labels)) > 1 and -1 not in set(dbscan_labels):
        metrics['DBSCAN_Silhouette'] = silhouette_score(X_scaled, dbscan_labels)
    
    # Informations structurelles
    metrics['DBSCAN_Noise_Count'] = list(dbscan_labels).count(-1)
    metrics['DBSCAN_Noise_Percentage'] = (list(dbscan_labels).count(-1) / len(dbscan_labels)) * 100
    
    return metrics, {'kmeans': kmeans_labels, 'dbscan': dbscan_labels}
```

## **Considérations Avancées et Recommandations**

### **Choix Algorithmique Contextuel**

**K-means - Privilégier quand** :
- Clusters sphériques et de tailles similaires
- Nombre de clusters connu a priori  
- Vitesse d'exécution critique
- Données sans outliers significatifs

**DBSCAN - Recommandé pour** :
- Formes de clusters arbitraires et complexes
- Détection d'outliers requise
- Densité variable dans l'espace des features
- Nombre de clusters inconnu ou variable

### **Standardisation Obligatoire**

**Pour DBSCAN** : La standardisation est **absolument critique**. Les différences d'échelles entre features créent des distorsions dans le calcul d'epsilon, pouvant complètement fausser la détection de densité.[1]

**Pour K-means** : Recommandée pour éviter la domination par les features à plus grande amplitude.

### **Limitations et Extensions**

**DBSCAN sans predict()** : Contrairement à K-means, DBSCAN ne possède pas de méthode `predict()` native. Pour classifier de nouveaux points, il faut **relancer l'algorithme complet** ou implémenter une logique de proximité aux core points existants.[1]

**Solutions alternatives** :
- **HDBSCAN** : Version hiérarchique gérant les densités variables
- **OPTICS** : Ordering Points To Identify Clustering Structure
- **Mean Shift** : Clustering basé sur la recherche de modes

## **Impact Pédagogique et Méthodologique**

Ce cours illustre parfaitement la progression naturelle d'apprentissage en machine learning : partir d'algorithmes conceptuellement simples (K-means) pour comprendre les limitations fondamentales, puis explorer des solutions plus sophistiquées (DBSCAN) répondant à ces limitations spécifiques.

L'approche d'optimisation de code démontre également une compétence cruciale en data science : **l'optimisation itérative**. Commencer par une solution fonctionnelle, identifier les goulots d'étranglement, puis appliquer des techniques d'optimisation ciblées.

### **Ressources Complémentaires Recommandées**

Le cours mentionne plusieurs ressources vidéo essentielles :[1]
- **3Blue1Brown** : Visualisations mathématiques des vecteurs propres (préparation PCA)
- **StatQuest** : Explications visuelles des concepts de clustering  
- **Free Calculus** : Fondements mathématiques

Ces ressources préparent efficacement aux concepts avancés de réduction de dimensionnalité qui suivront dans le curriculum.

[1](https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/attachments/76884410/49c1568b-4c18-4f98-82fb-95454117be10/transcriptM07D02.txt)
[2](https://perso.univ-lyon1.fr/marc.buffat/COURS/BOOK_OUTILSNUM_HTML/MGC2367M/cours/Chap2_Optimisation.html)
[3](https://www.reddit.com/r/learnpython/comments/11d6o3e/how_to_make_nested_for_loops_run_faster/)
[4](https://www.geeksforgeeks.org/python/counters-in-python-set-1/)
[5](https://www.9raytifclick.com/cours/les-collections-en-python/)
[6](https://koor.fr/Python/API/python/collections/Counter/Index.wp)
[7](https://opendatascience.com/unsupervised-learning-evaluating-clusters/)
[8](https://www.geeksforgeeks.org/machine-learning/elbow-method-for-optimal-value-of-k-in-kmeans/)
[9](https://www.geeksforgeeks.org/machine-learning/what-is-silhouette-score/)
[10](https://en.wikipedia.org/wiki/Silhouette_(clustering))
[11](https://www.geeksforgeeks.org/machine-learning/dbscan-clustering-in-ml-density-based-clustering/)
[12](https://laxmikants.github.io/blog/neural-network-using-make-moons-dataset/)
[13](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html)
[14](https://en.wikipedia.org/wiki/DBSCAN)
[15](https://3d.bk.tudelft.nl/courses/geo5017/slides/03_slides_Clustering.pdf)
[16](https://www.sefidian.com/2022/12/18/how-to-determine-epsilon-and-minpts-parameters-of-dbscan-clustering/)
[17](https://blog.dailydoseofds.com/p/how-to-find-optimal-epsilon-value)
[18](https://stackoverflow.com/questions/26666367/scikit-dbscan-eps-and-min-sample-value-determination)
[19](https://fr.linkedin.com/advice/1/how-can-you-optimize-performance-using-groupby-function-oxwbf?lang=fr)
[20](https://www.reddit.com/r/learnpython/comments/zoz4z1/speeding_up_pandas_groupbyapply_or_looping/)
[21](https://moncoachdata.com/blog/10-astuces-pour-accelerer-mes-scripts-python/)
[22](https://stackoverflow.com/questions/47392758/improving-the-performance-of-pandas-groupby)
[23](https://fr.linkedin.com/advice/1/how-can-you-optimize-game-loops-python-better-ycebc?lang=fr&lang=fr)
[24](https://docs.python.org/fr/3.8/library/collections.html)
[25](https://moncoachdata.com/blog/efficacite-et-performance-de-pandas/)
[26](https://www.docstring.fr/blog/4-facons-doptimiser-votre-code-python/)
[27](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/python-counter/)
[28](https://fr.linkedin.com/advice/0/how-do-you-troubleshoot-slow-groupby-operations-pandas-d7jtc?lang=fr)
[29](https://blog.stephane-robert.info/docs/developper/programmation/python/linting/)
[30](https://www.docstring.fr/blog/le-module-collections/)
[31](https://www.youtube.com/watch?v=VPbqZNM7wTI)
[32](https://www.data-bird.co/blog/boucle-for-python)
[33](https://tainix.fr/code/Gestion-d-occurrences-avec-Counter-en-Python)
[34](https://www.activeloop.ai/resources/glossary/radius-nearest-neighbors/)
[35](https://scikit-learn-extra.readthedocs.io/en/stable/modules/cluster.html)
[36](https://scikit-learn.org/0.16/modules/generated/sklearn.datasets.make_moons.html)
[37](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)
[38](https://python-course.eu/machine-learning/artificial-datasets-with-scikit-learn.php)
[39](https://scikit-learn.org/stable/modules/neighbors.html)
[40](https://www.kaggle.com/code/macespinoza/make-moons-python)
[41](https://arxiv.org/abs/2212.07679)
[42](https://datascientest.com/machine-learning-clustering-dbscan)
[43](https://gist.github.com/Mehdi-Amine/2a57f9317068ba754dd1bba7b44ba94f)
[44](https://en.wikipedia.org/wiki/Nearest_neighbor_search)
[45](https://www.kaggle.com/code/ghazouanihaythem/dbscan-tutorial)
[46](https://www.kaggle.com/code/nikhil25803/neural-network-make-moon-datasets)